In [18]:
import pandas as pd

# Leer el CSV
df = pd.read_csv('scores.csv')

# Función para crear y guardar una tabla pivote para cada característica
def create_and_save_comparison(df, feature_name):
    pivot = df.pivot_table(
        index=['model', feature_name],
        columns='prompt_strategy',
        values='mean_similarity',
        aggfunc='first'
    ).round(2)
    
    pivot['diff'] = (pivot['cot'] - pivot['zeroshot']).round(2)
    pivot['mean_similarity'] = pivot[['cot', 'zeroshot']].mean(axis=1).round(2)
    
    # Ordenar por modelo y promedio
    pivot = pivot.sort_values(['model', 'mean_similarity'])
    
    # Guardar a CSV
    filename = f'comparacion_por_mean_similarity_{feature_name}.csv'
    pivot.to_csv(filename)
    
    return pivot

# Crear un CSV para cada característica
layout_comparison = create_and_save_comparison(df, 'layout')
arrow_comparison = create_and_save_comparison(df, 'type_arrow')
label_comparison = create_and_save_comparison(df, 'type_label')
color_comparison = create_and_save_comparison(df, 'type_color')

# Mostrar las tablas
print("Comparación por Layout:")
print(layout_comparison)
print("\nComparación por Tipo de Flecha:")
print(arrow_comparison)
print("\nComparación por Tipo de Label:")
print(label_comparison)
print("\nComparación por Color:")
print(color_comparison)

Comparación por Layout:
prompt_strategy                          cot  zeroshot  diff  mean_similarity
model                         layout                                         
claude-3.5-sonnet             spectral  0.71      0.74 -0.03             0.72
                              kamada    0.86      0.87 -0.01             0.86
                              spring    0.87      0.88 -0.01             0.88
                              shell     0.93      0.89  0.04             0.91
                              circular  0.91      0.92 -0.01             0.92
                              spiral    0.93      0.94 -0.01             0.94
                              random    0.96      0.97 -0.01             0.96
gemini-pro-1.5                spectral  0.72      0.72  0.00             0.72
                              spring    0.92      0.93 -0.01             0.92
                              kamada    0.93      0.93  0.00             0.93
                              circular  

In [17]:
import pandas as pd

# Leer el CSV
df = pd.read_csv('comparacion_por_layout.csv')

# Crear una tabla pivote para COT
cot_table = df.pivot(
    index='model',
    columns='layout',
    values='cot'
).round(2)
print(cot_table)
# Crear una tabla pivote para zeroshot
zeroshot_table = df.pivot(
    index='model',
    columns='layout',
    values='zeroshot'
).round(2)

# Añadir sufijo a las columnas para distinguir entre COT y zeroshot
cot_table.columns = [f'{col}_cot' for col in cot_table.columns]
zeroshot_table.columns = [f'{col}_zeroshot' for col in zeroshot_table.columns]

# Combinar las dos tablas
final_table = pd.concat([cot_table, zeroshot_table], axis=1)

# Ordenar las columnas para que COT y zeroshot de cada layout estén juntos
all_layouts = df['layout'].unique()
ordered_columns = []
for layout in all_layouts:
    ordered_columns.extend([f'{layout}_cot', f'{layout}_zeroshot'])

final_table = final_table[ordered_columns]

# Guardar en CSV
final_table.to_csv('layout_comparison_final.csv')

print(final_table)

layout                         circular  kamada  random  shell  spectral  \
model                                                                      
claude-3.5-sonnet                  0.91    0.86    0.96   0.93      0.71   
gemini-pro-1.5                     0.93    0.93    0.96   0.94      0.72   
gpt-4o-2024-08-06                  0.87    0.90    0.84   0.84      0.77   
llama-3.2-90b-vision-instruct      0.88    0.62    0.66   0.90      0.61   
pixtral-12b                        0.76    0.88    0.87   0.65      0.71   
qwen-2-vl-72b-instruct             0.81    0.87    0.86   0.82      0.80   

layout                         spiral  spring  
model                                          
claude-3.5-sonnet                0.93    0.87  
gemini-pro-1.5                   0.94    0.92  
gpt-4o-2024-08-06                0.88    0.93  
llama-3.2-90b-vision-instruct    0.89    0.65  
pixtral-12b                      0.86    0.80  
qwen-2-vl-72b-instruct           0.50    0.85  
       

In [15]:
import pandas as pd
import plotly.graph_objects as go

# Leer los datos
df = pd.read_csv('comparacion_por_layout.csv')


# Crea una lista ordenada de layouts únicos (esto es la clave)
df_layouts = sorted(df['layout'].unique())


# Crear el gráfico
fig = go.Figure()

# Obtener los modelos únicos
modelos = df['model'].unique()

# Añadir una traza para cada modelo

colors = [
    'rgba(255, 0, 0, 0.5)',    # Rojo
    'rgba(0, 255, 0, 0.5)',    # Verde
    'rgba(0, 0, 255, 0.5)',    # Azul
    '#f1c40f',  # Amarillo
    'rgba(255, 0, 255, 0.5)',  # Magenta
    'rgba(0, 255, 255, 0.5)'   # Cian
]
dash_styles = ['solid', 'dash', 'dashdot', 'dot', 'longdash', 'longdashdot']
simbolos = ['circle', 'square', 'star', 'pentagon', 'x', 'star-triangle-up', 'cross']

for i, modelo in enumerate(modelos):
    df_modelo = df[(df['model'] == modelo)]

    # Ordena df_modelo según la lista ordenada de layouts
    df_modelo = df_modelo.set_index('layout').reindex(df_layouts).reset_index()

    df_modelo = pd.concat([df_modelo, df_modelo.iloc[[0]]])
    fig.add_trace(go.Scatterpolar(
        r=df_modelo['mean_similarity'],
        theta=df_modelo['layout'],
        #fill='toself',
        opacity=0.9,
        name=modelo,
        #line_width=6,
        line=dict(shape='spline', smoothing=0.0, color=colors[i], width=6),  # Líneas curvas
        #mode='lines+markers',
         marker=dict(
            color="#3b3b3b", symbol=simbolos[i],  size=15
        )
    ))
# Actualizar el diseño
fig.update_layout(
    title=dict(
        text="LVLM Type Comparison by Layout (Closer to 1 is Better)",
        font=dict(
            size=24,  # Tamaño del título
            family="Arial",  # Tipo de fuente
            color="black",   
        ),
    ),
    polar=dict(
         bgcolor="white",
         angularaxis=dict(
            gridcolor="gray",  # Color de las líneas angulares
            gridwidth=2,
            griddash="solid",
            tickfont=dict(size=27),    # Estilo de las líneas angulares
        ),
        radialaxis=dict(
            gridcolor="gray",  # Color de las líneas radiales
            gridwidth=2,
            griddash="solid",
            range=[min(df_modelo['mean_similarity']),1],
            dtick=0.05,  # Incremento de los ticks
            tickfont=dict(size=24),  # Estilo de las líneas radiales
        )
    ),
    legend=dict(
        font=dict(size=24),  # Tamaño del texto de la leyenda
    ),
 
)

fig.show()
fig.write_image("imagen.jpg", width=1200, height=900, scale=2)
